In [3]:
!pip install git+https://github.com/huggingface/transformers.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-qg4m3i2b
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-qg4m3i2b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [4]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch
torch.cuda.is_available()

True

In [5]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cuda:0


In [35]:
models = {
    'EleutherAI/gpt-neo-125M': 'EleutherAI/gpt-neo-125M',
    "EleutherAI/gpt-neo-1.3B": "EleutherAI/gpt-neo-1.3B",
    "KoboldAI/fairseq-dense-1.3B": "KoboldAI/fairseq-dense-1.3B",
    "2.7": "KoboldAI/fairseq-dense-2.7B-Nerys"
}
chosen_model = "EleutherAI/gpt-neo-1.3B"

In [36]:
tokenizer = AutoTokenizer.from_pretrained(chosen_model)

In [ ]:
# Download model
model = AutoModelForCausalLM.from_pretrained(chosen_model).to(device)

In [ ]:
# Load model to device:

# model = model.to(device)

In [ ]:
prompt = "Vyse and Aika are attacking a valuan military transport ship."
world_info = [
              "Valua is the capital city of the Valuan empire.",
              "Vyse and Aika are pirates who only attack military targets.",
]
genres = [
          "Pirate"
]
tokens_to_feed = 200
tokens_to_generate = 50
story_size = 1500

In [ ]:
def generate_text(prompt, tokens_to_generate, tokens_to_feed):
  input = ""
  
  input += prompt
  input_ids = tokenizer.encode(input, return_tensors="pt").to(device)
  print('input: ', input)
  print(tokens_to_generate+tokens_to_feed)
  gen_tokens = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.9,
    max_length=tokens_to_generate+tokens_to_feed,
  )
  gen_text = tokenizer.batch_decode(gen_tokens)[0]
  print('gen_text', gen_text)
  print("---------------")
  return gen_text
  

In [ ]:
def generate_story(prompt, world_info, genres, tokens_to_feed, tokens_to_generate, story_size):
  story = ""
  first_input = ""
  rounds = story_size / tokens_to_generate
  rounds = 5
  round = 0
  while round < rounds:
    input = ""
    if first_input == "":
      print("setup, len(genres):", len(genres), " len(world_info): ", len(world_info))
      if len(genres) > 0:
        for genre in genres:
          input += genre+". "
      if len(world_info) > 0:
        for info in world_info:
          input += info+" "
      input += story
      first_input = input
    else:
      if len(genres) > 0:
        for genre in genres:
          input += genre+" "
      if len(world_info) > 0:
        for info in world_info:
          input += info+" "
      input += story[len(story) - 200 : len(story)]

    generated_text = generate_text(input, tokens_to_generate, tokens_to_feed)
    story += generated_text[len(input) : len(generated_text)]
    round += 1
  return first_input + " " + story

In [ ]:
story = generate_story(prompt, world_info, genres, tokens_to_feed, tokens_to_generate, story_size)
print("Full story: ")
print(story)